In [ ]:
import yaml

with open('src/modules/news_summary/tickers.yaml', 'r') as file:
    file = yaml.safe_load(file)

# Test function
for a, b in file.items():
    print(a, b)

Ethereum_(ETH) https://news.google.com/search?q=etherium%20(ETH)&hl=en-US&gl=US&ceid=US%3Aen
Tether_(USDT) https://news.google.com/search?q=Tether%20(USDT)&hl=en-US&gl=US&ceid=US%3Aen
Binance_Coin_(BNB) https://news.google.com/search?q=Binance%20Coin%20(BNB)&hl=en-US&gl=US&ceid=US%3Aen
USD_Coin_(USDC) https://news.google.com/search?q=USD%20Coin%20(USDC)&hl=en-US&gl=US&ceid=US%3Aen
Solana_(SOL) https://news.google.com/search?q=Solana%20(SOL)&hl=en-US&gl=US&ceid=US%3Aen
XRP_(XRP) https://news.google.com/search?q=XRP%20(XRP)&hl=en-US&gl=US&ceid=US%3Aen
Cardano_(ADA) https://news.google.com/search?q=Cardano%20(ADA)&hl=en-US&gl=US&ceid=US%3Aen
Dogecoin_(DOGE) https://news.google.com/search?q=Dogecoin%20(DOGE)&hl=en-US&gl=US&ceid=US%3Aen
TRON_(TRX) https://news.google.com/search?q=TRON%20(TRX)&hl=en-US&gl=US&ceid=US%3Aen
Toncoin_(TON) https://news.google.com/search?q=Toncoin%20(TON)&hl=en-US&gl=US&ceid=US%3Aen
Polygon_(MATIC) https://news.google.com/search?q=Polygon%20(MATIC)&hl=en-US&gl=US&

In [94]:
import requests
from bs4 import BeautifulSoup

def scrape(
        url: str,
        number_headlines: int,
        printi: bool) -> list:
    """
    Scrape data from Google News
    """
    html_content = requests.get(url).content
    soup = BeautifulSoup(html_content, 'html.parser')
    headlines = soup.find_all('a', class_='JtKRv')
    all_headlines = [i.text for i in headlines]

    if printi:
        print(all_headlines)

    return all_headlines[:number_headlines]

# Test function
scrape_data = scrape(
    url=file["Solana_(SOL)"],
    number_headlines=30,
    printi=True
)

['Bitcoin rally lifts Solana, Coinbase, and other crypto plays', 'Crypto Price Analysis 12-4 BITCOIN: BTC, ETHEREUM: ETH, SOLANA: SOL, TONCOIN: TON, APTOS: APT, ARBITRUM: ARB, JUPITER: JUP', 'Comeback 2025: Is Ethereum poised to catch up with Bitcoin and Solana?', 'Missing Out on These 4 Cryptos Is Like Passing Up the Chance to Buy Solana (SOL) at $1.50', 'Trump’s crypto czar is Solana bull and Multicoin Capital investor', 'Solana: Ethereum Killer or Bot Playground? Diving Into the Numbers Behind the Surging Metrics', 'Solana Web3.js Library Compromised in Targeted Supply Chain Attack', 'Indicators Point to Massive Upswings for Dogecoin (DOGE) and This New DeFi Coin as Solana (SOL) Eyes $500 |', 'Grayscale files for Solana ETF as SOL token surges 6%', 'Solana ETF Pushes The Price: SOL’s Path To Future Growth', 'Grayscale files with SEC for spot Solana ETF', 'Solana Primed To Rally After Underperforming XRP and Cardano, According to Glassnode Co-Founders – Here’s Why', 'Analyst Jelle Sh

In [ ]:
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from datetime import datetime

import os
from dotenv import load_dotenv
load_dotenv('src/.env')


llm = ChatOpenAI(api_key=os.getenv("OPENAI_API_KEY"), model="gpt-4-turbo")
current_date = datetime.now().strftime("%Y-%m-%d-%H")

def summarizer(
        prompt_path: str,
        llm: ChatOpenAI,
        headlines: list,
        ticker: str,
        current_date: datetime,
        print_out: bool
    ):
    "Summarize Google News data"
    with open(prompt_path, 'r', encoding='utf-8') as file:
        markdown_string = file.read()
    prompt_template = PromptTemplate(template=markdown_string, input_variables=["headlines", "ticker", "current_date"])
    chain = prompt_template | llm | StrOutputParser()
    llm_output = chain.invoke({"headlines": headlines, "ticker": ticker, "current_date": current_date})

    if print_out:
        print(llm_output)

    return llm_output

# Test function

import json
out = summarizer(
    prompt_path="src/prompts/news_summarizer.md",
    llm=llm,
    headlines=scrape_data,
    ticker=list(file.keys())[4],
    current_date=current_date,
    print_out=True)

In [ ]:
import sys

for key, value in file.items():

    print(key, value)

    scrape_data = scrape(
        url=file[key],
        number_headlines=30,
        printi=True
    )

    out = summarizer(
        prompt_path="src/prompts/news_summarizer.md",
        llm=llm,
        headlines=scrape_data,
        ticker=value,
        current_date=current_date,
        print_out=False
    )


    print(out)


KeyboardInterrupt: 

In [105]:
import sys
sys.stdout = sys.__stdout__  # Restore the default standard output